In [1]:
import csv
import os
import traceback
import numpy as np
import tqdm
import pandas as pd
import multiprocessing as mp
import gc

import rsa_data_summary as rd
import rsa_data_wim as wim
import rsa_headers as rh
import config
import queries as q
import tools

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
files = tools.getfiles(r'C:\FTP\Syntell\SMEC RSA Files_GP PRM Sites_Dec21toFeb22')

COLLECTING FILES......


In [3]:
for file in files:
    df = tools.to_df(file)
    H = rh.Headers(df, file)
    header = H.header
    lanes = H.lanes
    data30 = df.loc[(df[0] == "30") & (df[1].isin(["0", "1", "2", "3", "4"]))].dropna(
            axis=1, how="all"
        )
    dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
        axis=1, how="all"
    ) 

    D = rd.Data(df, header)
    data = pd.DataFrame(columns=config.DATA_COLUMN_NAMES)

    d2 = D.dtype21
    if d2 is None:
        pass
    else:
        # d2.to_sql("electronic_count_data_type_21",
        #         con=config.ENGINE,
        #         schema="trafc",
        #         if_exists="append",
        #         index=False,
        #         method=tools.psql_insert_copy,
        #     )
        data = pd.concat([data, d2], keys=["site_id", "start_datetime", "lane_number"], ignore_index=True)

    d2 = D.dtype30
    if d2 is None:
        pass
    else:
        # d2.to_sql("electronic_count_data_type_30",
        #         con=config.ENGINE,
        #         schema="trafc",
        #         if_exists="append",
        #         index=False,
        #         method=tools.psql_insert_copy,
        #     )
        data = pd.concat([data, d2], keys=["site_id", "start_datetime", "lane_number"], ignore_index=True)

    d2 = D.dtype60
    if d2 is None:
        pass
    else:
        # d2.to_sql("electronic_count_data_type_60",
        #         con=config.ENGINE,
        #         schema="trafc",
        #         if_exists="append",
        #         index=False,
        #         method=tools.psql_insert_copy,
        #     )
        data = pd.concat([data, d2], keys=["site_id", "start_datetime", "lane_number"], ignore_index=True)

    d2 = D.dtype70
    if d2 is None:
        pass
    else:
        # d2.to_sql("electronic_count_data_type_70",
        #         con=config.ENGINE,
        #         schema="trafc",
        #         if_exists="append",
        #         index=False,
        #         method=tools.psql_insert_copy,
        #     )
        data = pd.concat([data, d2], keys=["site_id", "start_datetime", "lane_number"], ignore_index=True)

    break

In [50]:
data = pd.DataFrame(columns=config.DATA_COLUMN_NAMES)

# d2 = D.dtype21
# if d2 is None:
#     pass
# else:
#     data = data.merge(
#         d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
#     )

# d2 = D.dtype30
# if d2 is None:
#     pass
# else:
#     data = data.merge(
#         d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
#     )

# d2 = D.dtype70
# if d2 is None:
#     pass
# else:
#     data["start_datetime"] = data["start_datetime"].astype("datetime64[ns]")
#     d2["start_datetime"] = d2["start_datetime"].astype("datetime64[ns]")
#     data = data.merge(
#         d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
#     )

In [60]:
data2 = pd.concat([data, D.dtype21], keys=["site_id", "start_datetime", "lane_number"], ignore_index=True)
# data2 = data.merge(D.dtype21, how="right", left_on=["site_id", "start_datetime", "lane_number"], right_on=["site_id", "start_datetime", "lane_number"])
# data2 = data.join(D.dtype30, on=["site_id", "start_datetime", "lane_number"])

In [43]:
data2 = pd.concat([data2.set_index(["site_id", "start_datetime", "lane_number"], D.dtype30.set_index(["site_id", "start_datetime", "lane_number"])], keys=["site_id", "start_datetime", "lane_number"])

In [45]:
data2 = pd.concat([data2, D.dtype70], keys=["site_id", "start_datetime", "lane_number"], ignore_index=True)

In [63]:
data2.head()

header_id  \
               site_id start_datetime             lane_number                                         
start_datetime 0108    2022-02-01 00:00:00.000000 1            001a8b3b-0d3b-4e0e-9b02-79f3827308ce   
                                                  2            001a8b3b-0d3b-4e0e-9b02-79f3827308ce   
                                                  3            001a8b3b-0d3b-4e0e-9b02-79f3827308ce   
                                                  4            001a8b3b-0d3b-4e0e-9b02-79f3827308ce   
                       2022-02-01 00:15:00.000000 1            001a8b3b-0d3b-4e0e-9b02-79f3827308ce   

                                                              "year"  \
               site_id start_datetime             lane_number          
start_datetime 0108    2022-02-01 00:00:00.000000 1              NaN   
                                                  2              NaN   
                                                  3              NaN   
                                                  4              NaN   
                       2022-02-01 00:15:00.000000 1              NaN   

                                                                             end_datetime  \
               site_id start_datetime             lane_number                               
start_datetime 0108    2022-02-01 00:00:00.000000 1            2022-02-01 00:15:00.000000   
                                                  2            2022-02-01 00:15:00.000000   
                                                  3            2022-02-01 00:15:00.000000   
                                                  4            2022-02-01 00:15:00.000000   
                       2022-02-01 00:15:00.000000 1            2022-02-01 00:30:00.000000   

                                                              duration_min  \
               site_id start_datetime             lane_number                
start_datetime 0108    2022-02-01 00:00:00.000000 1                     15   
                                                  2                     15   
                                                  3                     15   
                                                  4                     15   
                       2022-02-01 00:15:00.000000 1                     15   

                                                              direction  \
               site_id start_datetime             lane_number             
start_datetime 0108    2022-02-01 00:00:00.000000 1                   P   
                                                  2                   P   
                                                  3                   N   
                                                  4                   N   
                       2022-02-01 00:15:00.000000 1                   P   

                                                              forward_direction_code  \
               site_id start_datetime             lane_number                          
start_datetime 0108    2022-02-01 00:00:00.000000 1                              NaN   
                                                  2                              NaN   
                                                  3                              NaN   
                                                  4                              NaN   
                       2022-02-01 00:15:00.000000 1                              NaN   

                                                              speedbin0  \
               site_id start_datetime             lane_number             
start_datetime 0108    2022-02-01 00:00:00.000000 1                   0   
                                                  2                   0   
                                                  3                   0   
                                                  4                   0   
                       2022-02-01 00:15:00.000000 1                   0

In [58]:
data2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001a8b3b-0d3b-4e0e-9b02-79f3827308ce,0108 Modderfontein Rd(CM),2022-02-01 00:15:00.000000,15,1,1,2,1,1,0,0,0,0,0,0,61,0,0,1,0,0,0,1,4,5,P,2022-02-01 00:00:00.000000,0108
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001a8b3b-0d3b-4e0e-9b02-79f3827308ce,0108 Modderfontein Rd(CM),2022-02-01 00:15:00.000000,15,2,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,4,4,P,2022-02-01 00:00:00.000000,0108
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001a8b3b-0d3b-4e0e-9b02-79f3827308ce,0108 Modderfontein Rd(CM),2022-02-01 00:15:00.000000,15,3,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,2,2,N,2022-02-01 00:00:00.000000,0108
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001a8b3b-0d3b-4e0e-9b02-79f3827308ce,0108 Modderfontein Rd(CM),2022-02-01 00:15:00.000000,15,4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,N,2022-02-01 00:00:00.000000,0108
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001a8b3b-0d3b-4e0e-9b02-79f3827308ce,0108 Modderfontein Rd(CM),2022-02-01 00:30:00.000000,15,1,0,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,P,2022-02-01 00:15:00.000000,0108


In [71]:
t2 = rd.Data.join(header, t1)

In [73]:
# t2.drop("station_name", axis=1, inplace=True)
# t2["start_datetime"] = t2["start_datetime"].astype("datetime64[ns]")
# d2["start_datetime"] = d2["start_datetime"].astype("datetime64[ns]")
# t2 = t2.merge(
#     d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
# )

In [64]:
data2.head()

,header_id_x,end_datetime_x_x,duration_min_x_x,lane_number,speedbin1_x,speedbin2_x,speedbin3_x,speedbin4_x,speedbin5_x,speedbin6_x,speedbin7_x,speedbin8_x,speedbin9_x,speedbin10_x,sum_of_heavy_vehicle_speeds_x,short_heavy_vehicles_x,medium_heavy_vehicles_x,long_heavy_vehicles_x,rear_to_rear_headway_shorter_than_2_seconds_x,rear_to_rear_headways_shorter_than_programmed_time_x,speedbin0_x,total_heavy_vehicles_type21_x,total_light_vehicles_type21_x,total_vehicles_type21_x,direction_x_x,start_datetime,site_id,end_datetime_y_x,duration_min_y_x,speedbin1_y,speedbin2_y,speedbin3_y,speedbin4_y,speedbin5_y,speedbin6_y,speedbin7_y,speedbin8_y,speedbin9_y,speedbin10_y,sum_of_heavy_vehicle_speeds_y,short_heavy_vehicles_y,medium_heavy_vehicles_y,long_heavy_vehicles_y,rear_to_rear_headway_shorter_than_2_seconds_y,rear_to_rear_headways_shorter_than_programmed_time_y,speedbin0_y,total_heavy_vehicles_type21_y,total_light_vehicles_type21_y,total_vehicles_type21_y,direction_y_x,header_id_y,end_datetime_x_y,duration_min_x_y,unknown_vehicle_error_class_x,light_motor_vehicles_x,heavy_vehicle_x,motorcycle_x,light_motor_vehicles_towing_x,two_axle_busses_x,two_axle_6_tyre_single_units_x,busses_with_3_or_4_axles_x,two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max_x,three_axle_single_unit_including_single_axle_light_trailer_x,four_or_less_axle_including_a_single_trailer_x,buses_with_5_or_more_axles_x,three_axle_single_unit_and_light_trailer_more_than_4_axles_x,five_axle_single_trailer_x,six_axle_single_trailer_x,five_or_less_axle_multi_trailer_x,six_axle_multi_trailer_x,seven_axle_multi_trailer_x,eight_or_more_axle_multi_trailer_x,total_light_vehicles_type30_x,total_heavy_vehicles_type30_x,total_vehicles_type30_x,direction_x_y,end_datetime_y_y,duration_min_y_y,unknown_vehicle_error_class_y,light_motor_vehicles_y,heavy_vehicle_y,motorcycle_y,light_motor_vehicles_towing_y,two_axle_busses_y,two_axle_6_tyre_single_units_y,busses_with_3_or_4_axles_y,two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max_y,three_axle_single_unit_including_single_axle_light_trailer_y,four_or_less_axle_including_a_single_trailer_y,buses_with_5_or_more_axles_y,three_axle_single_unit_and_light_trailer_more_than_4_axles_y,five_axle_single_trailer_y,six_axle_single_trailer_y,five_or_less_axle_multi_trailer_y,six_axle_multi_trailer_y,seven_axle_multi_trailer_y,eight_or_more_axle_multi_trailer_y,total_light_vehicles_type30_y,total_heavy_vehicles_type30_y,total_vehicles_type30_y,direction_y_y
0,c6e8b704-5dd8-400c-8f6c-f52198907a2c,2022-02-01 00:15:00.000000,15.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,2022-02-01 00:00:00,0350,2022-02-01 00:15:00,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,P,c6e8b704-5dd8-400c-8f6c-f52198907a2c,2022-02-01 00:15:00.000000,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,2022-02-01 00:15:00,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,P
1,c6e8b704-5dd8-400c-8f6c-f52198907a2c,2022-02-01 00:15:00.000000,15.0,2,0.0,0.0,4.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,243.0,0.0,0.0,3.0,0.0,1.0,0.0,3.0,4.0,7.0,N,2022-02-01 00:00:00,0350,2022-02-01 00:15:00,15,0,0,4,1,0,2,0,0,0,0,243,0,0,3,0,1,0,3,4,7,N,c6e8b704-5dd8-400c-8f6c-f52198907a2c,2022-02-01 00:15:00.000000,15.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,N,2022-02-01 00:15:00,15,0,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,4,N
2,c6e8b704-5dd8-400c-8f6c-f52198907a2c,2022-02-01 00:30:00.000000,15.0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,84.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,P,2022-02-01 00:15:00,0350,2022-02-01 00:30:00,15,0,0,0,1,0,0,0,0,0,0,84,1,0,0,0,0,0,1,0,1,P,c6e8b704-5dd8-400c-8f6c-f52198907a2c,2022-02-01 00:30:00.000000,15.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P,2022-02-01 00:30:00,15,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,P
3,c6e8b704-5dd8-400c-8f6c-f52198907a2c,2022-02-01 00:30:00.000000,15.0,2,0.0,0.0,1.

In [27]:
data.head()

,site_id,header_id,"""year""",start_datetime,end_datetime,duration_min,direction,forward_direction_code,lane_number,speedbin0,speedbin1,speedbin2,speedbin3,speedbin4,speedbin5,speedbin6,speedbin7,speedbin8,speedbin9,speedbin10,sum_of_heavy_vehicle_speeds,short_heavy_vehicles,medium_heavy_vehicles,long_heavy_vehicles,rear_to_rear_headway_shorter_than_2_seconds,rear_to_rear_headways_shorter_than_programmed_time,total_light_vehicles_type21,total_heavy_vehicles_type21,total_vehicles_type21,unknown_vehicle_error_class,motorcycle,light_motor_vehicles,light_motor_vehicles_towing,heavy_vehicle,two_axle_busses,two_axle_6_tyre_single_units,busses_with_3_or_4_axles,two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max,three_axle_single_unit_including_single_axle_light_trailer,four_or_less_axle_including_a_single_trailer,buses_with_5_or_more_axles,three_axle_single_unit_and_light_trailer_more_than_4_axles,five_axle_single_trailer,six_axle_single_trailer,five_or_less_axle_multi_trailer,six_axle_multi_trailer,seven_axle_multi_trailer,eight_or_more_axle_multi_trailer,total_light_vehicles_type30,total_heavy_vehicles_type30,total_vehicles_type30,number_of_error_vehicles,total_free_flowing_light_vehicles,total_following_light_vehicles,total_free_flowing_heavy_vehicles,total_following_heavy_vehicles,sum_of_inverse_of_speeds_for_free_flowing_lights,sum_of_inverse_of_speeds_for_following_lights,sum_of_inverse_of_speeds_for_free_flowing_heavies,sum_of_inverse_of_speeds_for_following_heavies,sum_of_speeds_for_free_flowing_lights,sum_of_speeds_for_following_lights,sum_of_speeds_for_free_flowing_heavies,sum_of_speeds_for_following_heavies,sum_of_squared_speeds_of_free_flowing_lights,sum_of_squared_speeds_for_following_lights,sum_of_squared_speeds_of_free_flowing_heavies,sum_of_squared_speeds_for_following_heavies,physical_lane_number,forward_1_or_reverse_code_2,vehicle_category,vehicle_class_code_primary_scheme,vehicle_class_code_secondary_scheme,vehicle_speed,vehicle_length,site_occupancy_time_in_milliseconds,chassis_height_code,vehicle_following_code,vehicle_tag_code,trailer_count,axle_count,bumper_to_1st_axle_spacing,sub_data_type_code_axle_spacing,number_of_axles_spacings_counted,axle_spacing_1_between_individual_axles_cm,axle_spacing_2_between_individual_axles_cm,axle_spacing_3_between_individual_axles_cm,axle_spacing_4_between_individual_axles_cm,axle_spacing_5_between_individual_axles_cm,axle_spacing_6_between_individual_axles_cm,axle_spacing_7_between_individual_axles_cm,axle_spacing_8_between_individual_axles_cm


In [ ]:


        # H = rh.Headers(df)
        # dfh = pd.DataFrame(
        #     df.loc[
        #         (df[0].isin(["H0", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1", "L2", "L3","L4" ,"L5","L6","L7","L8","L9","L10","L11","L12"]))
        #         | (
        #             (df[0].isin(["21", "70", "30", "13", "60"]))
        #             & (~df[1].isin(["0", "1", "2", "3", "4"]))
        #         )
        #         | (
        #             (df[0].isin(["10"]))
        #             & (df[1].isin(["1", "8", "5", "01", "08", "05"]))
        #         )
        #     ]
        # ).dropna(axis=1, how="all")
        # dfh["index"] = dfh.index
        # breaks = dfh["index"].diff() != 1
        # groups = breaks.cumsum()
        # dfh["newindex"] = groups
        # dfh = dfh.set_index("newindex")
        # dfh = dfh.drop(columns=["index"])
        # header = rh.header
        # header["document_url"] = str(file)


        # DATA = rd.Data(df)
        # data = tools.data_join(data, header)
        # data.drop("station_name", axis=1, inplace=True)

        # d2 = tools.dtype30(df)
        # if d2 is None:
        #     pass
        # else:
        #     d2 = tools.data_join(d2, header)
        #     data = data.merge(
        #         d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
        #     )

        # d2 = tools.dtype70(df)
        # if d2 is None:
        #     pass
        # else:
        #     data = tools.data_join(d2, header)
        #     data.drop("station_name", axis=1, inplace=True)
        #     data["start_datetime"] = data["start_datetime"].astype("datetime64[ns]")
        #     d2["start_datetime"] = d2["start_datetime"].astype("datetime64[ns]")
        #     data = data.merge(
        #         d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
        #     )

        # d3, sub_data_df = tools.dtype10(df)
        # if d3 is None:
        #     pass
        # else:
        #     data = tools.data_join(d3, header)
        #     tools.push_to_db(d3,
        #     "electronic_count_data_type_10",
        #     ["site_id", "start_datetime", "assigned_lane_number"],
        #     )

        #     sub_data_df = sub_data_df.replace(r'^\s*$', np.NaN, regex=True)
        #     sub_data_df = sub_data_df.drop("index", axis=1) 
        #     wx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'w']
        #     sx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 's']
        #     gx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'g']
        #     vx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'v']
        #     tx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 't']
        #     ax_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'a']
        #     cx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'c']

        #     if wx_data.empty:
        #         pass
        #     else:
        #         wx_data.rename(columns = {"value":"wheel_mass", "number":"wheel_mass_number", "id":"type10_id"}, inplace=True)
        #         # wx_data.to_sql(
        #         #     "traffic_e_type10_wheel_mass",
        #         #     con=config.ENGINE,
        #         #     schema="trafc",
        #         #     if_exists="append",
        #         #     index=False,
        #         #     method=tools.psql_insert_copy,
        #         # )

        #     if ax_data.empty:
        #         pass
        #     else:
        #         ax_data.rename(columns = {"value":"axle_mass", "number":"axle_mass_number", "id":"type10_id"}, inplace=True)
        #     #     ax_data.to_sql(
        #     #         "traffic_e_type10_axle_mass",
        #     #         con=config.ENGINE,
        #     #         schema="trafc",
        #     #         if_exists="append",
        #     #         index=False,
        #     #         method=tools.psql_insert_copy,
        #     #     )

        #     if gx_data.empty:
        #         pass
        #     else:
        #         gx_data.rename(columns = {"value":"axle_group_mass", "number":"axle_group_mass_number", "id":"type10_id"}, inplace=True)
        #     #     gx_data.to_sql(
        #     #         "traffic_e_type10_axle_group_mass",
        #     #         con=config.ENGINE,
        #     #         schema="trafc",
        #     #         if_exists="append",
        #     #         index=False,
        #     #         method=tools.psql_insert_copy,
        #     #     )

        #     if sx_data.empty:
        #         pass
        #     else:
        #         sx_data.rename(columns = {"value":"axle_spacing_cm", "number":"axle_spacing_number", "id":"type10_id"}, inplace=True)
        #         sx_data = sx_data.drop(["offset_sensor_detection_code","mass_measurement_resolution_kg"], axis=1)
        #     #     sx_data.to_sql(
        #     #         "traffic_e_type10_axle_spacing",
        #     #         con=config.ENGINE,
        #     #         schema="trafc",
        #     #         if_exists="append",
        #     #         index=False,
        #     #         method=tools.psql_insert_copy,
        #     #     )

        #     if tx_data.empty:
        #         pass
        #     else:
        #         tx_data.rename(columns = {"value":"tyre_code", "number":"tyre_number", "id":"type10_id"}, inplace=True)
        #         tx_data = tx_data.drop(["offset_sensor_detection_code","mass_measurement_resolution_kg"], axis=1)
        #     #     tx_data.to_sql(
        #     #         "traffic_e_type10_tyre",
        #     #         con=config.ENGINE,
        #     #         schema="trafc",
        #     #         if_exists="append",
        #     #         index=False,
        #     #         method=tools.psql_insert_copy,
        #     #     )

        #     if cx_data.empty:
        #         pass
        #     else:
        #         cx_data.rename(columns = {"value":"group_axle_count", "number":"group_axle_number", "id":"type10_id"}, inplace=True)
        #         cx_data = cx_data.drop(["offset_sensor_detection_code","mass_measurement_resolution_kg"], axis=1)
        #     #     cx_data.to_sql(
        #     #         "traffic_e_type10_axle_group_configuration",
        #     #         con=config.ENGINE,
        #     #         schema="trafc",
        #     #         if_exists="append",
        #     #         index=False,
        #     #         method=tools.psql_insert_copy,
        #     #     )

        #     if vx_data.empty:
        #         pass
        #     else:
        #         vx_data.rename(columns = {"value":"group_axle_count", "offset_sensor_detection_code":"vehicle_registration_number" ,"number":"group_axle_number", "id":"type10_id"}, inplace=True)
        #         vx_data = vx_data.drop(["mass_measurement_resolution_kg"], axis=1)
        #     #     vx_data.to_sql(
        #     #         "traffic_e_type10_identification_data_images",
        #     #         con=config.ENGINE,
        #     #         schema="trafc",
        #     #         if_exists="append",
        #     #         index=False,
        #     #         method=tools.psql_insert_copy,
        #     #     )

        # d2 = tools.dtype60(df)
        # if d2 is None:
        #     pass
        # else:
        #     data = tools.data_join(d2, header)
        #     data.drop("station_name", axis=1, inplace=True)
        #     data = data.merge(
        #         d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
        #     )

        # data = data.rename(columns=(lambda x: x[:-2] if '_x' in x else x))
        # header = header.rename(columns=(lambda x: x[:-2] if '_x' in x else x))

        # header = tools.header_calcs(header, data, 21)
        # header = tools.header_calcs(header, data, 30)
        # header = tools.header_calcs(header, data, 70)
        # header = tools.header_calcs(header, data, 60)

        # data = data[data.columns.intersection(config.DATA_COLUMN_NAMES)]
        # header = header[header.columns.intersection(config.HEADER_COLUMN_NAMES)]

        # # tools.push_to_partitioned_table(
        # #     data,
        # #     "electronic_count_data_partitioned",
        # #     ["site_id", "start_datetime", "lane_number"],
        # # )

        # # tools.push_to_db(
        # #     header,
        # #     "electronic_count_header",
        # #     ["site_id", "start_datetime", "end_datetime"],
        # # )

        # data.to_csv(r"C:\Users\MB2705851\Desktop\Temp\rsa_traffic_counts\data.csv", index=False, mode='a')
        # header.to_csv(r"C:\Users\MB2705851\Desktop\Temp\rsa_traffic_counts\header.csv", index=False, mode='a')

    #     with open(
    #         os.path.expanduser(config.FILES_COMPLETE),
    #         "a",
    #         newline="",
    #     ) as f:
    #         write = csv.writer(f)
    #         write.writerows([[files]])


In [15]:
print(df)

       0       1       2         3       4       5        6       7       8   \
0      H0     200     002       RSA    Data  Format  Version    2.00    None   
1      S0    0013    0013      Paul  Kruger     Ext     None    None    None   
2      I0  RT8010     Fam  140426MA    None    None     None    None    None   
3      D0       M       L      None    None    None     None    None    None   
4      D1  211201  000000    211231  240000  211020   113132  211130  230005   
...    ..     ...     ...       ...     ...     ...      ...     ...     ...   
35722  30       0  211231      2400      15      04     0000    0002    0000   
35723  70       0  211231      2400      15      01     0007    0000    0002   
35724  70       0  211231      2400      15      02     0011    0000    0000   
35725  70       0  211231      2400      15      03     0011    0000    0000   
35726  70       0  211231      2400      15      04     0002    0000    0000   

           9         10        11      